In [1]:
import torch
import torchaudio
import datasets
from datasets import load_dataset, load_metric
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import re


/sysadmin/wirawan/miniconda3/envs/bert2bert/lib/python3.7/site-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '


In [2]:
# Preprocessing the datasets.
# We need to read the audio files as arrays
def speech_file_to_array_fn(batch):
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower()
    batch["sentence"] = batch["sentence"].replace('！ ', '')
    batch["sentence"] = batch["sentence"].replace('，', '')
    batch["sentence"] = batch["sentence"].replace('é', 'e')
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    batch["speech"] = resampler(speech_array).squeeze().numpy()
    return batch

In [3]:
# Preprocessing the datasets.
# We need to read the aduio files as arrays
def evaluate(batch):
    inputs = processor(batch["speech"], sampling_rate=16_000, return_tensors="pt", padding=True)

    with torch.no_grad():
        logits = model(inputs.input_values.to("cuda"), attention_mask=inputs.attention_mask.to("cuda")).logits

    pred_ids = torch.argmax(logits, dim=-1)
    batch["pred_strings"] = processor.batch_decode(pred_ids)
    return batch

In [6]:
dataset = datasets.load_from_disk("dataset_sundanese")

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['path', 'sentence'],
        num_rows: 3791
    })
    test: Dataset({
        features: ['path', 'sentence'],
        num_rows: 422
    })
})

In [8]:
#ds = dataset.train_test_split(test_size=0.1, seed=1)
common_voice_train = dataset['train']
common_voice_test = dataset['test']

In [9]:
common_voice_train, common_voice_test

(Dataset({
     features: ['path', 'sentence'],
     num_rows: 3791
 }),
 Dataset({
     features: ['path', 'sentence'],
     num_rows: 422
 }))

In [10]:
test_dataset = common_voice_test
wer = load_metric("wer")

In [11]:
model_name = "./wav2vec2-large-xlsr-sundanese"
#model_name = "cahya/wav2vec2-large-xlsr-indonesian"
#model_name = "ayameRushia/wav2vec2-large-xlsr-indonesia-demo"
#model_name = "Galuh/wav2vec2-large-xlsr-indonesian-demo"

processor = Wav2Vec2Processor.from_pretrained(model_name)
model = Wav2Vec2ForCTC.from_pretrained(model_name) 
model.to("cuda")

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureExtractor(
      (conv_layers): ModuleList(
        (0): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        )
        (1): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        )
        (2): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        )
        (3): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        )
        (4): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,)

In [12]:
#chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\'\”]'
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\'\”_\�]'
resampler = torchaudio.transforms.Resample(48_000, 16_000)
test_dataset = test_dataset.map(speech_file_to_array_fn)

In [13]:
result = test_dataset.map(evaluate, batched=True, batch_size=8)

print("WER: {:2f}".format(100 * wer.compute(predictions=result["pred_strings"], references=result["sentence"])))


WER: 6.190727
